# 单机多卡训练

随着深度学习的发展，模型和数据集越来越大，有时单张显卡无法满足训练任务的显存要求，或者单卡训练用时太久，影响训练速度，这些情况下需要用到多卡训练的方式。飞桨框架 2.0 增加 [paddle.distributed.spawn](../api/paddle/distributed/spawn_cn.html) 函数来启动单机多卡训练，同时原有的 [paddle.distributed.launch](../api/paddle/distributed/launch_cn.html) 的方式依然保留。

## 一、launch启动

### 1.1 高层API场景

当调用 [paddle.Model](../api/paddle/Model_cn.html) 高层API来实现训练时，想要启动单机多卡训练非常简单，代码不需要做任何修改，只需要在启动时增加一下参数 `-m paddle.distributed.launch` 。
以MNIST为例，使用高层API的训练代码如下：

In [ ]:
import paddle
import numpy as np
from paddle.vision.transforms import ToTensor

# 加载训练数据集和测试数据集
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=ToTensor())
test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=ToTensor())

# 使用 Sequential 模型组网
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(1, -1), 
    paddle.nn.Linear(784, 512), 
    paddle.nn.ReLU(), 
    paddle.nn.Dropout(0.2), 
    paddle.nn.Linear(512, 10)
)

# 使用 paddle.Model 封装模型
model = paddle.Model(mnist)

# 使用 Model.prepare 配置训练准备参数
model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters()), 
              loss=paddle.nn.CrossEntropyLoss(), 
              metrics=paddle.metric.Accuracy())

# 使用 Model.fit 训练模型
model.fit(train_dataset, 
          epochs=5, 
          batch_size=64,
          verbose=1)

将上述代码保存为train.py，使用高层API启动多卡训练的命令如下：

In [ ]:
# 单机单卡启动，默认使用第0号卡
! python train.py
# 单机多卡启动，默认使用当前可见的所有卡
! python -m paddle.distributed.launch train.py
# 单机多卡启动，设置当前使用的第0号和第1号卡
! python -m paddle.distributed.launch --gpus='0,1' train.py
# 单机多卡启动，设置当前使用第0号和第1号卡
! export CUDA_VISIBLE_DEVICES=0,1
! python -m paddle.distributed.launch train.py

这里补充一段介绍这个方式启动后发生了什么？任务怎么分配到不同卡上的
另外针对这里应该会有常见的问题定位流程，补充一下介绍，有FAQ可以补一下到FAQ的链接。

（待补充）

### 1.2 基础API场景

如果使用基础API实现现训练，想要启动单机多卡训练，需要对单机单卡的代码进行3处修改，具体如下：


In [ ]:
import paddle
from paddle.vision.transforms import ToTensor
# 第1处改动，导入分布式训练所需的包
import paddle.distributed as dist
# 加载数据集
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=ToTensor())
test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=ToTensor())
# 第2处改动，初始化并行环境
dist.init_parallel_env()

# 定义网络结构
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(1, -1),
    paddle.nn.Linear(784, 512),
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(512, 10)
)
# 用 DataLoader 实现数据加载
train_loader = paddle.io.DataLoader(train_dataset, batch_size=32, shuffle=True)

# 第3处改动，增加paddle.DataParallel封装
mnist = paddle.DataParallel(mnist)
mnist.train()
# 设置迭代次数
epochs = 5
# 设置优化器
optim = paddle.optimizer.Adam(parameters=mnist.parameters())
for epoch in range(epochs):
    for batch_id, data in enumerate(train_loader()):
        x_data = data[0]            # 训练数据
        y_data = data[1]            # 训练数据标签
        predicts = mnist(x_data)    # 预测结果
        # 计算损失 等价于 prepare 中loss的设置
        loss = paddle.nn.functional.cross_entropy(predicts, y_data)
        # 计算准确率 等价于 prepare 中metrics的设置
        acc = paddle.metric.accuracy(predicts, y_data)
        # 下面的反向传播、打印训练信息、更新参数、梯度清零都被封装到 Model.fit() 中
        # 反向传播
        loss.backward()
        if (batch_id+1) % 1800 == 0:
            print("epoch: {}, batch_id: {}, loss is: {}, acc is: {}".format(epoch, batch_id, loss.numpy(), acc.numpy()))
        # 更新参数
        optim.step()
        # 梯度清零
        optim.clear_grad()

修改完后保存文件为train.py，然后使用跟高层API相同的启动方式即可。

补充：

这里基础API实现的效果和高层一模一样吗？完全没有差异？有没有基础API可以更灵活应用的场景？为什么高层不用补额外的配置代码？


In [ ]:
# 单机多卡启动，默认使用当前可见的所有卡
! python -m paddle.distributed.launch train.py
# 单机多卡启动，设置当前使用的第0号和第1号卡
! python -m paddle.distributed.launch --gpus '0,1' train.py
# 单机多卡启动，设置当前使用第0号和第1号卡
! export CUDA_VISIBLE_DEVICES=0,1
! python -m paddle.distributed.launch train.py

## 二、spawn启动

 `launch` 方式启动训练，以文件为单位启动多进程，需要用户在启动时调用 `paddle.distributed.launch` ，对于进程的管理要求较高。飞桨框架2.0版本增加了 `spawn` 启动方式，可以更好地控制进程，在日志打印、训练退出时更友好。
 
（补充“对进程的管理要求较高”、“可以更好地控制进程，在日志打印、训练退出时更友好”这几句话的理解）

### 2.1 高层API场景

使用 `spawn` 方式启动多卡训练时，需要先将训练的过程封装成一个函数，将超参数设为该函数的参数传入训练流程中。代码如下所示：

In [ ]:
import paddle
import numpy as np
from paddle.vision.transforms import ToTensor
# 高层API场景使用spwan方式时，需要导入paddle.distributed包
import paddle.distributed as dist

def train():
    # 加载训练数据集和测试数据集
    train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=ToTensor())
    test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=ToTensor())

    # 使用 Sequential 模型组网
    mnist = paddle.nn.Sequential(
        paddle.nn.Flatten(1, -1), 
        paddle.nn.Linear(784, 512), 
        paddle.nn.ReLU(), 
        paddle.nn.Dropout(0.2), 
        paddle.nn.Linear(512, 10)
    )

    # 使用 paddle.Model 封装模型
    model = paddle.Model(mnist)

    # 使用 Model.prepare 配置训练准备参数
    model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters()), 
                  loss=paddle.nn.CrossEntropyLoss(), 
                  metrics=paddle.metric.Accuracy())

    # 使用 Model.fit 训练模型
    model.fit(train_dataset, 
              epochs=5, 
              batch_size=64,
              verbose=1)


# 传入训练函数，指定进程数并指定当前使用的卡号
# （这里我测试使用多卡会报错，只能单卡跑）
if __name__ == '__main__':
    dist.spawn(train, nprocs=1, gpus='0')

### 2.2 基础API场景

与高层API场景类似，使用 `spawn` 方式启动多卡训练时，需要先将训练的过程封装成一个函数，将超参数设为该函数的参数传入训练流程中。同时，也需要与 `paddle.distributed.launch` 过程类似，进行三处改动：导入分布式包、初始化并行环境和将模型封装。具体代码如下：

In [ ]:
from __future__ import print_function

import paddle
import paddle.nn as nn
import paddle.optimizer as opt
# 第1处改动，导入分布式训练所需的包
import paddle.distributed as dist

def train(print_result=False):
    # 加载数据集
    train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=ToTensor())
    test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=ToTensor())
    # 第2处改动，初始化并行环境
    dist.init_parallel_env()

    # 定义网络结构
    mnist = paddle.nn.Sequential(
        paddle.nn.Flatten(1, -1),
        paddle.nn.Linear(784, 512),
        paddle.nn.ReLU(),
        paddle.nn.Dropout(0.2),
        paddle.nn.Linear(512, 10)
    )
    # 用 DataLoader 实现数据加载
    train_loader = paddle.io.DataLoader(train_dataset, batch_size=32, shuffle=True)

    # 第3处改动，增加paddle.DataParallel封装
    mnist = paddle.DataParallel(mnist)
    mnist.train()
    # 设置迭代次数
    epochs = 5
    # 设置优化器
    optim = paddle.optimizer.Adam(parameters=mnist.parameters())
    for epoch in range(epochs):
        for batch_id, data in enumerate(train_loader()):
            x_data = data[0]            # 训练数据
            y_data = data[1]            # 训练数据标签
            predicts = mnist(x_data)    # 预测结果
            # 计算损失 等价于 prepare 中loss的设置
            loss = paddle.nn.functional.cross_entropy(predicts, y_data)
            # 计算准确率 等价于 prepare 中metrics的设置
            acc = paddle.metric.accuracy(predicts, y_data)
            # 下面的反向传播、打印训练信息、更新参数、梯度清零都被封装到 Model.fit() 中
            # 反向传播
            loss.backward()
            if (batch_id+1) % 1800 == 0 and print_reslut:
                print("epoch: {}, batch_id: {}, loss is: {}, acc is: {}".format(epoch, batch_id, loss.numpy(), acc.numpy()))
            # 更新参数
            optim.step()
            # 梯度清零
            optim.clear_grad()

# 传入训练函数、参数、指定进程数并指定当前使用的卡号
if __name__ == '__main__':
    dist.spawn(train, args=(True,), nprocs=2, gpus='4,5')

调用 [paddle.distributed.spawn](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/distributed/spawn_cn.html) 来启动多卡训练时，可根据需要设置参数：
* func：由 spawn 方法启动的进程所调用的目标函数。
* args：传入目标函数 func 的参数。
* nprocs：启动进程的数目。当仅需要使用部分可见的GPU设备进行训练时，可设置该参数指定GPU数。例如：当前机器有8张GPU卡 {0,1,2,3,4,5,6,7}，此时会使用前两张卡 {0,1}；或者当前机器通过配置环境变量 CUDA_VISIBLE_DEVICES=4,5,6,7，仅使4张GPU卡可见，此时会使用可见的前两张卡 {4,5}。若不设置该参数，默认使用所有可见的GPU设备训练。
* gpus：指定训练使用的GPU ID。例如 gpus='4,5' 可指定使用第4号卡和第5号卡。若不设置该参数，默认使用GPU ID序号较小的GPU。

# 三、总结

待补充